# **Welcome to the ABDICO clustering Notebook**#

* This notebook helps recognize dominant groups of related actors, objects, regulated actions ("aims") and modals ("deontics") in institutional statements.
Compared to traditional word based topic modeling such as LDA, we use BERTopic, which pursues a semantic ('word meaning') based approach.

### ***This notebook performs the following tasks***


* It takes a main.csv file which has columns designated as "Attribute", "Objects", "Deontics" and "Aims" respectively
* User indicates the institutional constituent (ABDICO) over which clustering is to be performed
* Output main.csv contains a "_group" column, indicating the respective topic cluster to which the constituent belongs.
* The categorical topic of the group is indicated by the top most representative words from the cluster.








# **Installations and Setup**
* This code sets up the analysis. You don't have to understand it. Just run it and then scroll down.
* These commands below install the necessary components for the rest of the analysis to work. To run press ***ctrl+enter*** keys or select ***Runtime*** from the menu above and then one of the ***Run*** options within it.

In [ ]:
!pip install transformers
!pip install sentence-transformers
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c8023e671cab18df8fc90fbb8a41a464f679607bbe843480d32440b8c7bda987
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.3 MB/

# **Data upload**

Run this cell to run your own search engine.

For your own data, you will likely have to adapt it for this notebook to run. See below for the sample format

Please name uploaded files as `main.csv` and `main2.csv`. For each institutional statement, there must be columns indicating the corresponding ABDICO codings. That is "Attribute", "Object", "Deontic", "Aim". These may be hancoded or computationally extracted.

See ABDICO_parsing.ipynb in this repository for automated institutional grammar coding of policies.


In [ ]:
from google.colab import files
uploaded = files.upload()

# **Or use our data archives**

You may also uncomment and run the cell below to follow the demonstration on archival data. This shall directly download (into Colab) our datasets for the Apache Software Foundation, an open source software community.

* [ABDICO coded Apache community policies](https://storage.googleapis.com/cscw_2022/anamika_os.csv)

In [ ]:
##Else you can directly use the !wget command below to download our datasets into the code notebook,
##make sure you uncomment the below code by (ctrl+/) keys before running this cell of code

## ABDICO coded policy example
# !wget -O main.csv https://storage.googleapis.com/routines_semantic/ASF_ABDICO.csv
# import pandas as pd
# import nltk
# nltk.download('punkt')
# data = pd.read_csv('main.csv')
# data['raw institutional statement'] = data['raw institutional statement'].apply(lambda x : nltk.tokenize.sent_tokenize(x))
# data = data.explode('raw institutional statement')
# data.to_csv('main.csv',index=False)

--2023-10-19 02:18:09--  https://storage.googleapis.com/routines_semantic/ASF_ABDICO.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.207, 108.177.98.207, 74.125.197.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61905 (60K) [text/csv]
Saving to: ‘main.csv’

main.csv            100%[===================>]  60.45K  --.-KB/s    in 0s      

2023-10-19 02:18:09 (133 MB/s) - ‘main.csv’ saved [61905/61905]



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#**Clustering and topic modeling**

* Prior to uploading the your own dataset remove all the NAN(not a number) text value rows from the csv file.
* You may need to make some changes to the first three lines of the following cell:

  * **Component** : The insitutional component of policies/institutional statements you would like to cluster and group. This should exactly match the column in which data which contains the ABDICO component of interest. Case Sensitive.
  * **top_n_words** : The N dominant words to indidate/represent the topic of the group. We recommend setting this number no more than 10.
  * **num_topic** : Now this one is a little tricky. This value guides the clustering alogithm to find the specified number of clusters and topics among the ABDICO components. You may use an estimate (e.g. if you are clustering attributes and are aware that there are around 5 different actors in your policy data), or set a slightly higher number. It's simpler to manually combine some similar clusters. Our default is 20, for our apache policies.

In [ ]:
Component = "Object"
top_n_words = 3
num_topic = 20

# clustering of components
from bertopic import BERTopic
import pandas as pd
result = pd.read_csv('main.csv', usecols=[Component])
result.dropna(subset=[Component],inplace=True)
components = result[Component].tolist()

topic_model = BERTopic(top_n_words = 5,nr_topics = num_topic)
topic_model.hdbscan_model.gen_min_span_tree=True
topic_model.umap_model.random_state= 0 ##set seed to enable reproduction of clustering

topic_model.fit(components)
freq = topic_model.get_topic_info()
result[Component + '_group'] = topic_model.transform(components)[0]
result[Component + '_group'] = result[Component + '_group'].apply(lambda x : freq.loc[x+1,'Name'])
result.to_csv('main.csv', index=False)
files.download('main.csv')

   Topic  Count                                    Name  \
0     -1     32              -1_what_they_found_checked   
1      0     16       0_champion_mentors_incubation_and   
2      1     12  1_disclaimer_disclaimers_progress_work   
3      2     15             2_asf_non_policies_releases   
4      3     18             3_apache_the_replace_effort   
5      4     12     4_releases_cut_available_conditions   
6      5     21            5_podling_podlings_on_voting   
7      6     52                 6_the_to_incubator_that   
8      7     23       7_community_diverse_to_committers   
9      8     11              8_vote_votes_line_approach   

                                    Representation  \
0                 [what, they, found, checked, to]   
1     [champion, mentors, incubation, and, mentor]   
2  [disclaimer, disclaimers, progress, work, text]   
3     [asf, non, policies, releases, successfully]   
4             [apache, the, replace, effort, name]   
5    [releases, cut, avail

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Going deeper
Check out [BERTopic](https://maartengr.github.io/BERTopic/) documentation for more arguments, parameters and methods for sophisticated topic modeling. For "fine tuning" your topic modeling, see our [work](https://arxiv.org/abs/2309.14245) on governance of open source software.